In [1]:
#Data Interpolation and creation of a Net CDF file for T&S Float data
"""This will interpolate T&S and BGC data to standard pressures defined 
    by the World Ocean Atlas.
        Those pressures are as follows, in db: 
        [5,10,20,30,50,75,100,125,150,200,250,300,400,500,600,700,800,900,
        1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000]"""

#Initialize packages and file folders
import numpy as np
import pandas as pd
from scipy import interpolate
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta
import requests
import time
import os
import glob
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Base filepath
root = '/global/homes/k/kefalc/code/'
#profile_dir = root + 'Profiles/'
TS_dir = root + 'Test_Data/'#Change this for other projects
BGC_dir= root + 'Test_Data_BGC/'#Change this for other projects

In [2]:
#Function to mask data, leaving only good data (QC 1 or 2)
#2 is "probably good"

def maskdata(BGC_QC_array, BGC_array, Pres_QC_array, Pres_array):
    #Mask BGC Array first
    mask = np.ma.masked_where(BGC_QC_array > 2, BGC_array)
    mask = np.ma.masked_where(BGC_QC_array == 0, BGC_array) 
    mask = np.ma.masked_where(Pres_array <= 0, BGC_array)
    
    param_masked_array = np.ma.compressed(mask) #masked array for chosen BGC parameter
    
    #Mask Pres array second
    mask = np.ma.masked_where(Pres_QC_array > 2, Pres_array)
    mask = np.ma.masked_where(Pres_QC_array == 0, Pres_array)
    mask = np.ma.masked_where(Pres_array <= 0, Pres_array)
    
    pres_masked_array = np.ma.compressed(mask) #masked array for pressure
    
    #Combine masked arrays into one array
    masked_values = np.stack((pres_masked_array, param_masked_array), axis=1)
    #remove all rows that contain NaNs
    masked_values = masked_values[~np.isnan(masked_values).any(axis=1), :]
    
    #order values based to go from lowest to highest based on the pressure for interpolation 
    masked_values=masked_values[masked_values[:, 0].argsort()]
    
    
    return masked_values

In [3]:
def pc_interpolation(pressure, parameter):
    interp_values = interpolate.pchip_interpolate(pressure, temp, pres_interp)
    return(interp_values)

In [4]:
#Create data for netCDF file

#Initialize data
temp = np.tile(np.NaN,(29, 300000)) #temperature
sal = np.tile(np.NaN,(29, 300000)) #salinity

#Initialize coordinates and dimensions #need to add attributes
wmoid = np.tile(np.NaN, 300000) #WMO ID Coordinate
time = np.tile(np.NaN, 300000) #Time, JULD, Coordinate
lat = np.tile(np.NaN, 300000) #latitude, Coordinate
lon = np.tile(np.NaN, 300000) #longitude, Coordinate
pres_interp=np.array([0,5,10,20,30,50,75,100,125,150,200,250,300,400,500,600,700,800,900,
        1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100, 2200]) #pressure, Dimension
pres_th = 0.1 #pressure threshold around levels to interpolate to
profile = np.tile(np.NaN, 300000) #Count of profiles, Dimension


k=0
#Load data
file_list = glob.glob(TS_dir + '*_prof.nc')

for filename in file_list: #add something to only open prof.nc files
    data = xr.open_dataset(filename)
    data = data.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    
    #get wmoid
    name, ext = os.path.splitext(filename)
    split_file = name.split('/')
    float_id, file_type = split_file[-1].split('_')
    
    for num in range(len(data.PROF_NUM)):
        #Pressure variables
        pres_qc = data['PRES_ADJUSTED_QC'].isel(PROF_NUM=num).values.astype(float).flatten()
        pres_adjusted= data['PRES_ADJUSTED'].isel(PROF_NUM=num).values.flatten()
        #Temperature variables
        temp_qc = data['TEMP_ADJUSTED_QC'].isel(PROF_NUM=num).values.astype(float).flatten()
        temp_adjusted= data['TEMP_ADJUSTED'].isel(PROF_NUM=num).values.flatten()
        #Salinity variables:
        sal_qc = data['PSAL_ADJUSTED_QC'].isel(PROF_NUM=num).values.astype(float).flatten()
        sal_adjusted = data['PSAL_ADJUSTED'].isel(PROF_NUM=num).values.astype(float).flatten()
        
        #Other data needed
        Jul_day = data['JULD'].isel(PROF_NUM=num).values.flatten() #time
        latitude = data['LATITUDE'].isel(PROF_NUM=num).values.flatten() #lat
        longitude = data['LONGITUDE'].isel(PROF_NUM=num).values.flatten() #lon

        #Temperature mask
        masked_temp = maskdata(temp_qc, temp_adjusted, pres_qc, pres_adjusted)
        #salinity mask
        masked_sal = maskdata(sal_qc, sal_adjusted, pres_qc, pres_adjusted)
        
       #delete duplicate pressure values
       #unique temperature
        (up_temp, u_index) = np.unique(masked_temp[:,0], return_index = True)
        
        u_temp = masked_temp[u_index, 1]
        
        unique_temp = np.stack((up_temp,u_temp), axis = 1)
        
        #unique salinity
        (up_sal, u_index) = np.unique(masked_sal[:,0], return_index = True)

        u_sal = masked_sal[u_index, 1]

        unique_sal = np.stack((up_sal,u_sal), axis = 1)
                
        #order data set so pressure is increasing
        unique_temp.sort(axis = 0)
        unique_sal.sort(axis = 0)
    
        #Remove any NaNs/infinite values that remain
        unique_temp[unique_temp == float('+inf')] = np.NaN
        unique_temp[unique_temp == float('-inf')] = np.NaN
        unique_temp = unique_temp[~np.isnan(unique_temp).any(axis=1), :]
        
        unique_sal[unique_sal == float('+inf')] = np.NaN
        unique_sal[unique_sal == float('-inf')] = np.NaN
        unique_sal = unique_sal[~np.isnan(unique_sal).any(axis=1), :]
        
        
        pressure_t = unique_temp[:,0] 
        temperature = unique_temp[:,1]
        pressure_s = unique_sal[:,0]
        salinity = unique_sal[:,1]
        
        
        parameter = min(len(salinity), len(temperature))
        
        #skip profiles that do not have sufficient data
        if parameter > 5:
            temp_interpolant = interpolate.PchipInterpolator(pressure_t, temperature, extrapolate = False)
            temp_interp_values = temp_interpolant(pres_interp)
            temp_interp_values = temp_interp_values[1:-2]
            
            sal_interpolant = interpolate.PchipInterpolator(pressure_s, salinity, extrapolate = False)
            sal_interp_values = sal_interpolant(pres_interp)
            sal_interp_values = sal_interp_values[1:-2]
            
            # Find where there are pressure measurements in each level bin, with a threshold around the levels
            val_mask = np.zeros((len(pres_interp)-1,))
            for levs in range(0,len(pres_interp)-1):
                val_mask[levs] = np.sum(((pressure_t)>= (pres_interp[levs]-pres_th))& (pressure_t<= (pres_interp[levs+1]+pres_th))) > 0

            # Create a mask of levels where pressures are in one bin above and two bins below    
            level_maskarray = (val_mask[0:-2] + val_mask[1:-1] + val_mask[2:]) == 3
            
            temp_interp_values[level_maskarray==0] = np.nan
            sal_interp_values[level_maskarray==0] = np.nan

            temp[:,k]=temp_interp_values
            sal[:,k]=sal_interp_values
            wmoid[k] = float_id
            time[k]= Jul_day
            lat[k] = latitude
            lon[k] = longitude
            profile[k] = k
            k=k+1
        
        else:
            continue

In [5]:
#Save data as an xarray data set
ds = xr.Dataset({'temperature': (['pressure', 'profile'],  temp, {'units': 'Degrees C'}),
                 'salinity': (['pressure', 'profile'], sal, {'units': 'psu'})
                },
                 coords={'lon': (['profile'], lon),
                         'lat': (['profile'], lat),
                         'time': (['profile'], time, {'units': 'Julian Day'}),
                         'profile': (['profile'], profile),
                         'WMOID': (['profile'], wmoid),
                         'pressure': (['pressure'], pres_interp[1:-2], {'units': 'decibar'})})

#ds.to_netcdf(root + 'conv_test.nc') 

In [6]:
#Remove empty profiles and convert lons
#ds = xr.open_dataset(root + 'southern_ocean_ts.nc')

ds = ds.dropna(dim="profile", how = "all")

for i in range(len(ds.lon)):
    if ds.lon[i] > 180.0:
        ds.lon[i] = ds.lon[i] - 360.0
    if ds.lon[i] <= -180.0:
        ds.lon[i] = ds.lon[i] + 360.0

display(ds)

#ds.to_netcdf(root + 'SO_ts_cut.nc') 
#NOTE: 127403 profiles

<xarray.Dataset>
Dimensions:      (pressure: 29, profile: 127398)
Coordinates:
    lon          (profile) float64 167.8 169.7 172.6 ... -51.5 -51.72 -50.98
    lat          (profile) float64 -56.03 -55.43 -54.94 ... -58.6 -59.1 -59.26
    time         (profile) float64 1.268e+18 1.269e+18 ... 1.617e+18 1.618e+18
  * profile      (profile) float64 0.0 1.0 2.0 ... 1.274e+05 1.274e+05 1.274e+05
    WMOID        (profile) float64 5.903e+06 5.903e+06 ... 5.904e+06 5.904e+06
  * pressure     (pressure) int64 5 10 20 30 50 75 ... 1600 1700 1800 1900 2000
Data variables:
    temperature  (pressure, profile) float64 nan nan nan nan ... nan nan nan nan
    salinity     (pressure, profile) float64 nan nan nan nan ... nan nan nan nan

In [ ]:
#save a single pressure level
ds = xr.open_dataset(root + 'southern_ocean_ts.nc')
#ds = ds.sel(pressure = slice(10,11))
ds = ds.dropna(dim="profile", how = "all")
display(ds)
#124251 profiles at pressurel level 10

#ds.to_netcdf(root + 'SO_ts_10.nc')

In [7]:
#Find indexes where lat and lon are within the desired range

lat = ds["lat"].to_index()
lon = ds["lon"].to_index()

index1 = []
index2 = []
for i in range(len(lat)):
    if lon[i] < -120: #note: leave 5-10 degrees extra for calculations.
        #print(lon[i])
        index1 = np.append(index1, i)
    if lon[i] > -150:
        index2 = np.append(index2, i)
lonindex = np.intersect1d(index1, index2)

#display(len(index1))
#display(len(index2))
#display(lonindex)
#display(len(lonindex)) 

lonindex = lonindex.astype(int)

ds_new = ds.isel(profile = lonindex)

In [8]:
#trim dataset based on latitude range

lat = ds_new["lat"].to_index()
lon = ds_new["lon"].to_index()

index1 = []
index2 = []
for i in range(len(lat)):
    if lat[i] < -40:
        index1 = np.append(index1, i)
        
    if lat[i] > -70:
        index2 = np.append(index2, i)
latindex = np.intersect1d(index1, index2)

#display(len(index1))
#display(len(index2))
#display(latindex)
#display(len(latindex))    

latindex = latindex.astype(int)

ds_new = ds_new.isel(profile = latindex)
display(ds_new)
#25909 profiles at this pressure level

<xarray.Dataset>
Dimensions:      (pressure: 29, profile: 30872)
Coordinates:
    lon          (profile) float64 -147.7 -146.9 -146.3 ... -123.4 -122.1 -120.9
    lat          (profile) float64 -50.89 -52.18 -52.12 ... -64.7 -64.9 -65.1
    time         (profile) float64 1.302e+18 1.303e+18 ... 1.536e+18 1.537e+18
  * profile      (profile) float64 39.0 40.0 41.0 ... 1.273e+05 1.273e+05
    WMOID        (profile) float64 5.903e+06 5.903e+06 ... 5.904e+06 5.904e+06
  * pressure     (pressure) int64 5 10 20 30 50 75 ... 1600 1700 1800 1900 2000
Data variables:
    temperature  (pressure, profile) float64 nan nan nan nan ... nan nan nan nan
    salinity     (pressure, profile) float64 nan nan nan nan ... nan nan nan nan

In [10]:
#save new trimmed dataset
ds_new.to_netcdf(root + 'SO_ts.nc')
#NOTE: 25909 profiles